In [32]:
import numpy as np
import pandas as pd
from models import Hankel,Rank,Cluster,Meepc
import warnings
warnings.simplefilter('ignore')
import math
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [33]:
hankel = Hankel()
rank = Rank()
cluster = Cluster()
meepc = Meepc.MEEPC()

In [34]:
df = pd.read_csv('~/data/ctown/dataset03.csv')
df

,DATETIME,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,06/01/14 00,0.509730,2.049003,3.191145,2.792634,2.656091,5.316831,1.562321,98.998444,1,...,87.605774,26.495605,84.206619,18.901676,81.983734,18.791777,67.125603,29.387470,28.487471,0
1,06/01/14 01,0.412580,2.009072,3.642565,2.831673,3.126387,5.494855,1.852043,99.095901,1,...,89.448341,26.487326,85.900085,18.849329,82.150589,18.739643,67.178696,29.354256,28.454256,0
2,06/01/14 02,0.320112,1.986093,4.140192,3.256733,3.574601,5.500000,2.246126,98.420959,1,...,91.056114,26.487364,86.582474,19.597170,83.988579,19.496712,72.425293,29.354538,28.454538,0
3,06/01/14 03,0.332879,2.009203,4.673478,3.744497,3.952379,5.500000,3.203573,97.575172,1,...,92.594353,26.575815,88.020546,26.028486,64.670486,25.922703,76.275040,29.449951,28.549952,0
4,06/01/14 04,0.483496,2.089049,5.237937,4.409456,3.504676,5.500000,4.439714,97.351059,1,...,94.473099,26.723457,90.422462,26.209970,64.746620,26.104692,76.703529,29.574265,28.674263,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,05/01/15 20,3.349998,3.393529,3.704372,3.318569,1.976920,5.500000,4.797106,96.594833,1,...,71.224411,28.940384,84.560669,21.076448,81.414070,20.970821,71.399384,31.532953,30.632954,0
8757,05/01/15 21,3.262380,3.738153,3.319207,3.162533,2.355615,5.207708,3.493474,96.445862,1,...,69.354080,29.247887,84.673790,26.999941,83.265991,26.999941,52.213036,31.836996,30.936996,0
8758,05/01/15 22,3.114685,3.948144,3.123156,3.060679,2.593729,4.997561,2.207609,96.929070,1,...,86.823753,28.358240,84.593727,20.571791,81.679344,20.457272,66.498741,31.281141,30.381142,0
8759,05/01/15 23,2.931601,3.892631,3.542954,3.025374,2.923712,5.104730,2.071863,97.206474,1,...,87.819832,28.297560,83.987007,20.450956,82.001610,20.337372,66.844086,31.210789,30.310789,0


In [35]:
df_1 = pd.read_csv('~/data/ctown/dataset04.csv')
df_normal=df_1[df_1['ATT_FLAG']==0]



In [36]:
df=pd.concat((df,df_normal),axis=0,ignore_index=True)
df

,DATETIME,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,06/01/14 00,0.509730,2.049003,3.191145,2.792634,2.656091,5.316831,1.562321,98.998444,1,...,87.605774,26.495605,84.206619,18.901676,81.983734,18.791777,67.125603,29.387470,28.487471,0
1,06/01/14 01,0.412580,2.009072,3.642565,2.831673,3.126387,5.494855,1.852043,99.095901,1,...,89.448341,26.487326,85.900085,18.849329,82.150589,18.739643,67.178696,29.354256,28.454256,0
2,06/01/14 02,0.320112,1.986093,4.140192,3.256733,3.574601,5.500000,2.246126,98.420959,1,...,91.056114,26.487364,86.582474,19.597170,83.988579,19.496712,72.425293,29.354538,28.454538,0
3,06/01/14 03,0.332879,2.009203,4.673478,3.744497,3.952379,5.500000,3.203573,97.575172,1,...,92.594353,26.575815,88.020546,26.028486,64.670486,25.922703,76.275040,29.449951,28.549952,0
4,06/01/14 04,0.483496,2.089049,5.237937,4.409456,3.504676,5.500000,4.439714,97.351059,1,...,94.473099,26.723457,90.422462,26.209970,64.746620,26.104692,76.703529,29.574265,28.674263,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12441,24/12/16 20,2.650000,2.370000,3.850000,3.040000,3.820000,4.940000,2.190000,120.080000,1,...,70.030000,27.380000,84.140000,18.450000,81.670000,18.340000,66.040000,29.880000,28.980000,0
12442,24/12/16 21,2.240000,2.560000,3.420000,2.920000,3.690000,5.020000,1.970000,119.120000,1,...,68.600000,27.660000,83.460000,25.400000,60.850000,25.280000,66.890000,30.190000,29.290000,0
12443,24/12/16 22,1.910000,2.760000,2.950000,2.490000,2.700000,5.140000,1.870000,120.710000,1,...,85.630000,26.840000,82.820000,24.460000,59.560000,24.340000,66.080000,29.680000,28.780000,0
12444,24/12/16 23,1.520000,2.520000,3.330000,2.030000,1.690000,5.100000,1.390000,120.020000,1,...,86.150000,25.780000,103.630000,24.770000,59.010000,24.650000,66.420000,28.980000,28.080000,0


In [37]:
df=df.drop(['DATETIME'],axis=1)
df_label=df.iloc[:,-1]
df=df.iloc[:,:-1]
df_colms=df.columns
scaler=StandardScaler()
df=scaler.fit_transform(df)
df=pd.DataFrame(df,columns=df_colms)
df

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J300,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422
0,-1.810652,-0.840399,-1.489684,-1.438753,-0.125827,-0.299891,-2.102859,-0.192322,0.0,0.701055,...,-0.485068,0.969072,-0.467118,0.198329,-1.258253,0.754636,-1.256119,-0.150254,-1.034333,-0.359477
1,-1.891658,-0.867565,-0.824829,-1.366330,0.513682,0.702713,-1.754244,-0.182644,0.0,0.703361,...,-0.486352,1.186900,-0.470150,0.423304,-1.270108,0.771354,-1.267867,-0.140255,-1.042174,-0.371926
2,-1.968762,-0.883198,-0.091921,-0.577770,1.123162,0.731687,-1.280055,-0.249670,0.0,0.687387,...,-0.484604,1.376971,-0.470137,0.513959,-1.100738,0.955519,-1.097261,0.847781,-1.042107,-0.371820
3,-1.958116,-0.867475,0.693507,0.327115,1.636865,0.731687,-0.127985,-0.333661,0.0,0.667368,...,-0.451086,1.558821,-0.437744,0.705006,0.355822,-0.980130,0.350843,1.572762,-1.019584,-0.336060
4,-1.832527,-0.813155,1.524846,1.560727,1.028078,0.731687,1.359430,-0.355917,0.0,0.662064,...,-0.395961,1.780926,-0.383674,1.024098,0.396924,-0.972501,0.391854,1.653455,-0.990238,-0.289469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12441,-0.026020,-0.622019,-0.519318,-0.979847,1.456856,-2.422136,-1.347590,1.901208,0.0,-1.641849,...,-0.123946,-1.108734,-0.143233,0.189478,-1.360548,0.723200,-1.357927,-0.354694,-0.918065,-0.174880
12442,-0.367893,-0.492759,-1.152625,-1.202467,1.280082,-1.971590,-1.612310,1.805874,0.0,-1.641849,...,-0.018536,-1.277789,-0.040691,0.099141,0.213483,-1.362938,0.206009,-0.194622,-0.844885,-0.058695
12443,-0.643058,-0.356696,-1.844844,-2.000190,-0.066119,-1.295771,-1.732637,1.963771,0.0,-1.641849,...,-0.360210,0.735496,-0.340993,0.014118,0.000592,-1.492194,-0.005821,-0.347161,-0.965277,-0.249839
12444,-0.968254,-0.519972,-1.285178,-2.853569,-1.439516,-1.521044,-2.310208,1.895249,0.0,-1.641849,...,-0.730962,0.796970,-0.729189,2.778709,0.070801,-1.547303,0.064038,-0.283132,-1.130522,-0.512194


In [38]:
df=pd.concat((df,df_label),axis=1)
df

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,-1.810652,-0.840399,-1.489684,-1.438753,-0.125827,-0.299891,-2.102859,-0.192322,0.0,0.701055,...,0.969072,-0.467118,0.198329,-1.258253,0.754636,-1.256119,-0.150254,-1.034333,-0.359477,0
1,-1.891658,-0.867565,-0.824829,-1.366330,0.513682,0.702713,-1.754244,-0.182644,0.0,0.703361,...,1.186900,-0.470150,0.423304,-1.270108,0.771354,-1.267867,-0.140255,-1.042174,-0.371926,0
2,-1.968762,-0.883198,-0.091921,-0.577770,1.123162,0.731687,-1.280055,-0.249670,0.0,0.687387,...,1.376971,-0.470137,0.513959,-1.100738,0.955519,-1.097261,0.847781,-1.042107,-0.371820,0
3,-1.958116,-0.867475,0.693507,0.327115,1.636865,0.731687,-0.127985,-0.333661,0.0,0.667368,...,1.558821,-0.437744,0.705006,0.355822,-0.980130,0.350843,1.572762,-1.019584,-0.336060,0
4,-1.832527,-0.813155,1.524846,1.560727,1.028078,0.731687,1.359430,-0.355917,0.0,0.662064,...,1.780926,-0.383674,1.024098,0.396924,-0.972501,0.391854,1.653455,-0.990238,-0.289469,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12441,-0.026020,-0.622019,-0.519318,-0.979847,1.456856,-2.422136,-1.347590,1.901208,0.0,-1.641849,...,-1.108734,-0.143233,0.189478,-1.360548,0.723200,-1.357927,-0.354694,-0.918065,-0.174880,0
12442,-0.367893,-0.492759,-1.152625,-1.202467,1.280082,-1.971590,-1.612310,1.805874,0.0,-1.641849,...,-1.277789,-0.040691,0.099141,0.213483,-1.362938,0.206009,-0.194622,-0.844885,-0.058695,0
12443,-0.643058,-0.356696,-1.844844,-2.000190,-0.066119,-1.295771,-1.732637,1.963771,0.0,-1.641849,...,0.735496,-0.340993,0.014118,0.000592,-1.492194,-0.005821,-0.347161,-0.965277,-0.249839,0
12444,-0.968254,-0.519972,-1.285178,-2.853569,-1.439516,-1.521044,-2.310208,1.895249,0.0,-1.641849,...,0.796970,-0.729189,2.778709,0.070801,-1.547303,0.064038,-0.283132,-1.130522,-0.512194,0


# --done with training data--

In [39]:
df_attack=pd.read_csv('/home/gajender/meepc/attackdata_labeledwithattacktype.csv')
df_attack

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG,AttackID
0,3.51,2.55,3.60,2.24,1.62,4.97,2.84,93.19,1,93.21,...,22.50,82.83,16.84,79.83,16.56,80.37,39.76,24.20,1,1
1,3.66,1.68,4.00,2.23,1.98,5.23,3.13,91.44,1,91.46,...,21.86,83.52,29.21,85.83,29.21,54.31,41.20,23.54,1,1
2,3.84,0.83,4.44,2.46,2.51,5.47,3.45,90.22,1,90.24,...,21.13,84.47,29.89,86.62,29.89,55.20,41.96,22.79,1,1
3,3.81,0.65,4.89,2.93,3.07,5.50,3.95,94.77,1,94.79,...,25.83,86.16,28.10,86.30,28.10,57.49,28.88,27.98,1,1
4,3.75,1.00,5.24,3.42,3.67,5.36,4.78,94.80,1,94.81,...,27.65,88.04,18.20,83.38,17.95,85.43,30.34,29.44,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,0.39,3.81,4.02,3.28,4.00,5.50,3.77,98.55,1,98.57,...,28.10,86.71,19.27,82.66,19.16,70.54,30.92,30.02,1,7
488,0.50,3.69,4.50,3.72,3.31,5.50,4.27,97.42,1,97.44,...,28.09,85.77,26.08,62.67,25.97,73.59,30.95,30.05,1,7
489,0.66,3.61,4.97,3.90,2.61,5.50,4.95,96.87,1,96.89,...,28.10,87.05,29.10,62.90,29.10,60.38,30.96,30.06,1,7
490,0.92,3.68,5.33,4.31,2.08,5.22,3.65,95.94,1,95.96,...,29.24,89.16,29.62,62.82,29.62,59.34,31.80,30.90,1,7


In [40]:
df_attack['AttackID'].value_counts()

7    110
4     94
6     94
3     60
5     60
1     50
2     24
Name: AttackID, dtype: int64

In [41]:
df_test=pd.read_csv('/home/gajender/meepc/testdata_labeledwithattacktype.csv')
df_test

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG,AttackID
0,0.73,2.27,4.00,3.26,3.87,5.50,4.28,98.93,1.0,98.95,...,26.74,84.52,19.43,83.27,19.33,71.33,29.61,28.71,0.0,-1
1,0.69,2.25,4.53,3.26,3.84,5.50,4.78,97.95,1.0,97.97,...,26.73,85.04,25.97,64.22,25.86,73.79,29.63,28.73,0.0,-1
2,0.90,2.31,5.03,3.41,3.32,5.16,3.22,96.82,1.0,96.84,...,26.89,87.16,29.18,63.81,29.18,59.05,29.80,28.90,0.0,-1
3,1.11,2.54,5.16,3.97,2.82,5.01,2.54,96.76,1.0,96.78,...,28.23,88.83,26.53,63.42,26.41,70.92,30.80,29.90,0.0,-1
4,1.27,2.99,4.94,4.27,2.35,5.38,3.41,94.77,1.0,94.79,...,31.32,69.55,27.46,63.43,27.34,70.88,33.61,32.71,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,1.94,2.09,3.67,2.95,1.74,4.96,2.09,120.00,1.0,0.00,...,27.15,84.05,18.28,80.39,18.17,65.97,29.66,28.76,0.0,-1
2085,1.54,2.27,3.24,2.84,2.07,5.02,1.84,122.23,1.0,0.00,...,27.08,83.12,17.04,79.84,16.93,66.14,29.53,28.63,0.0,-1
2086,1.07,2.24,2.99,2.30,2.34,5.12,1.78,122.44,1.0,0.00,...,26.20,83.77,16.97,80.25,16.86,66.29,28.98,28.08,0.0,-1
2087,0.85,2.14,3.44,2.35,2.67,5.24,1.82,98.71,1.0,98.73,...,26.59,84.07,19.00,81.37,18.89,66.07,29.49,28.59,0.0,-1


In [42]:
test_colms=df_test.columns

In [43]:
test_colms

Index(['L_T1', 'L_T2', 'L_T3', 'L_T4', 'L_T5', 'L_T6', 'L_T7', 'F_PU1',
       'S_PU1', 'F_PU2', 'S_PU2', 'F_PU3', 'S_PU3', 'F_PU4', 'S_PU4', 'F_PU5',
       'S_PU5', 'F_PU6', 'S_PU6', 'F_PU7', 'S_PU7', 'F_PU8', 'S_PU8', 'F_PU9',
       'S_PU9', 'F_PU10', 'S_PU10', 'F_PU11', 'S_PU11', 'F_V2', 'S_V2',
       'P_J280', 'P_J269', 'P_J300', 'P_J256', 'P_J289', 'P_J415', 'P_J302',
       'P_J306', 'P_J307', 'P_J317', 'P_J14', 'P_J422', 'ATT_FLAG',
       'AttackID'],
      dtype='object')

In [44]:
df_test['AttackID'].value_counts()

-1     1682
 12     100
 13      80
 8       70
 9       65
 10      31
 11      31
 14      30
Name: AttackID, dtype: int64

In [45]:
df_test=pd.concat((df_attack,df_test),axis=0,ignore_index=True)
df_test

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG,AttackID
0,3.51,2.55,3.60,2.24,1.62,4.97,2.84,93.19,1.0,93.21,...,22.50,82.83,16.84,79.83,16.56,80.37,39.76,24.20,1.0,1
1,3.66,1.68,4.00,2.23,1.98,5.23,3.13,91.44,1.0,91.46,...,21.86,83.52,29.21,85.83,29.21,54.31,41.20,23.54,1.0,1
2,3.84,0.83,4.44,2.46,2.51,5.47,3.45,90.22,1.0,90.24,...,21.13,84.47,29.89,86.62,29.89,55.20,41.96,22.79,1.0,1
3,3.81,0.65,4.89,2.93,3.07,5.50,3.95,94.77,1.0,94.79,...,25.83,86.16,28.10,86.30,28.10,57.49,28.88,27.98,1.0,1
4,3.75,1.00,5.24,3.42,3.67,5.36,4.78,94.80,1.0,94.81,...,27.65,88.04,18.20,83.38,17.95,85.43,30.34,29.44,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2576,1.94,2.09,3.67,2.95,1.74,4.96,2.09,120.00,1.0,0.00,...,27.15,84.05,18.28,80.39,18.17,65.97,29.66,28.76,0.0,-1
2577,1.54,2.27,3.24,2.84,2.07,5.02,1.84,122.23,1.0,0.00,...,27.08,83.12,17.04,79.84,16.93,66.14,29.53,28.63,0.0,-1
2578,1.07,2.24,2.99,2.30,2.34,5.12,1.78,122.44,1.0,0.00,...,26.20,83.77,16.97,80.25,16.86,66.29,28.98,28.08,0.0,-1
2579,0.85,2.14,3.44,2.35,2.67,5.24,1.82,98.71,1.0,98.73,...,26.59,84.07,19.00,81.37,18.89,66.07,29.49,28.59,0.0,-1


In [46]:
df_test.reset_index(inplace=True)

In [47]:
df_test

,index,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,...,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG,AttackID
0,0,3.51,2.55,3.60,2.24,1.62,4.97,2.84,93.19,1.0,...,22.50,82.83,16.84,79.83,16.56,80.37,39.76,24.20,1.0,1
1,1,3.66,1.68,4.00,2.23,1.98,5.23,3.13,91.44,1.0,...,21.86,83.52,29.21,85.83,29.21,54.31,41.20,23.54,1.0,1
2,2,3.84,0.83,4.44,2.46,2.51,5.47,3.45,90.22,1.0,...,21.13,84.47,29.89,86.62,29.89,55.20,41.96,22.79,1.0,1
3,3,3.81,0.65,4.89,2.93,3.07,5.50,3.95,94.77,1.0,...,25.83,86.16,28.10,86.30,28.10,57.49,28.88,27.98,1.0,1
4,4,3.75,1.00,5.24,3.42,3.67,5.36,4.78,94.80,1.0,...,27.65,88.04,18.20,83.38,17.95,85.43,30.34,29.44,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2576,2576,1.94,2.09,3.67,2.95,1.74,4.96,2.09,120.00,1.0,...,27.15,84.05,18.28,80.39,18.17,65.97,29.66,28.76,0.0,-1
2577,2577,1.54,2.27,3.24,2.84,2.07,5.02,1.84,122.23,1.0,...,27.08,83.12,17.04,79.84,16.93,66.14,29.53,28.63,0.0,-1
2578,2578,1.07,2.24,2.99,2.30,2.34,5.12,1.78,122.44,1.0,...,26.20,83.77,16.97,80.25,16.86,66.29,28.98,28.08,0.0,-1
2579,2579,0.85,2.14,3.44,2.35,2.67,5.24,1.82,98.71,1.0,...,26.59,84.07,19.00,81.37,18.89,66.07,29.49,28.59,0.0,-1


In [48]:
y_actual=df_test.iloc[:,-1]
attacklabels=hankel.fit(np.asarray(y_actual),60,0.5)
attacklabels

array([[ 1,  1,  2, ..., 14, -1, -1],
       [ 1,  1,  2, ..., 14, -1, -1],
       [ 1,  1,  2, ..., -1, -1, -1],
       ...,
       [ 2,  3,  3, ..., -1, -1, -1],
       [ 2,  3,  3, ..., -1, -1, -1],
       [ 2,  3,  3, ..., -1, -1, -1]])

In [49]:
attacklabels_df=pd.DataFrame(attacklabels)

In [50]:
attacklabels.shape

(60, 85)

In [51]:
isattack = np.any(attacklabels>0,axis=0)
attackindices=np.where(isattack)[0]
attackindices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       25, 26, 27, 28, 36, 37, 38, 39, 44, 45, 46, 47, 48, 56, 57, 58, 59,
       60, 67, 68, 69, 70, 71, 80, 81, 82])

In [52]:
np.unique(attacklabels[:,0])

array([1, 2])

In [53]:
attack_dict={}
for idx in attackindices:
    attack_dict[idx]=np.unique(attacklabels[:,idx])
attack_dict

{0: array([1, 2]),
 1: array([1, 2, 3]),
 2: array([2, 3]),
 3: array([3, 4]),
 4: array([3, 4]),
 5: array([4]),
 6: array([4, 5]),
 7: array([4, 5]),
 8: array([5, 6]),
 9: array([5, 6]),
 10: array([6]),
 11: array([6, 7]),
 12: array([6, 7]),
 13: array([7]),
 14: array([7]),
 15: array([-1,  7]),
 16: array([-1,  7]),
 25: array([-1,  8]),
 26: array([-1,  8]),
 27: array([-1,  8]),
 28: array([-1,  8]),
 36: array([-1,  9]),
 37: array([-1,  9]),
 38: array([-1,  9]),
 39: array([-1,  9]),
 44: array([-1, 10]),
 45: array([-1, 10]),
 46: array([-1, 10, 11]),
 47: array([-1, 11]),
 48: array([-1, 11]),
 56: array([-1, 12]),
 57: array([-1, 12]),
 58: array([12]),
 59: array([-1, 12]),
 60: array([-1, 12]),
 67: array([-1, 13]),
 68: array([-1, 13]),
 69: array([13]),
 70: array([-1, 13]),
 71: array([-1, 13]),
 80: array([-1, 14]),
 81: array([-1, 14]),
 82: array([-1, 14])}

In [54]:
def find_threshold(radii_n,radii_a):
    label = [1]*len(radii_n) + [-1]*len(radii_a)
    label = np.array(label)
    radiis = np.array(list(radii_n)+list(radii_a))
    indices = np.argsort(radiis)
    label = label[indices]
    pos_temp = (1+label)//2
    neg_temp = (1-label)//2
    tp = np.cumsum(pos_temp)
    fp = np.cumsum(neg_temp)
    fn = tp[-1] - tp
    fmeas = 2*tp / (2*tp + fp + fn)
    # print(fmeas)
    idx = np.argmax(fmeas)
    # print(indices[idx])
    return radiis[indices[idx]]

In [55]:
df_normal.columns

Index(['DATETIME', 'L_T1', 'L_T2', 'L_T3', 'L_T4', 'L_T5', 'L_T6', 'L_T7',
       'F_PU1', 'S_PU1', 'F_PU2', 'S_PU2', 'F_PU3', 'S_PU3', 'F_PU4', 'S_PU4',
       'F_PU5', 'S_PU5', 'F_PU6', 'S_PU6', 'F_PU7', 'S_PU7', 'F_PU8', 'S_PU8',
       'F_PU9', 'S_PU9', 'F_PU10', 'S_PU10', 'F_PU11', 'S_PU11', 'F_V2',
       'S_V2', 'P_J280', 'P_J269', 'P_J300', 'P_J256', 'P_J289', 'P_J415',
       'P_J302', 'P_J306', 'P_J307', 'P_J317', 'P_J14', 'P_J422', 'ATT_FLAG'],
      dtype='object')

In [56]:
df_test.columns

Index(['index', 'L_T1', 'L_T2', 'L_T3', 'L_T4', 'L_T5', 'L_T6', 'L_T7',
       'F_PU1', 'S_PU1', 'F_PU2', 'S_PU2', 'F_PU3', 'S_PU3', 'F_PU4', 'S_PU4',
       'F_PU5', 'S_PU5', 'F_PU6', 'S_PU6', 'F_PU7', 'S_PU7', 'F_PU8', 'S_PU8',
       'F_PU9', 'S_PU9', 'F_PU10', 'S_PU10', 'F_PU11', 'S_PU11', 'F_V2',
       'S_V2', 'P_J280', 'P_J269', 'P_J300', 'P_J256', 'P_J289', 'P_J415',
       'P_J302', 'P_J306', 'P_J307', 'P_J317', 'P_J14', 'P_J422', 'ATT_FLAG',
       'AttackID'],
      dtype='object')

In [57]:
df_normal=df
df_normal

,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,-1.810652,-0.840399,-1.489684,-1.438753,-0.125827,-0.299891,-2.102859,-0.192322,0.0,0.701055,...,0.969072,-0.467118,0.198329,-1.258253,0.754636,-1.256119,-0.150254,-1.034333,-0.359477,0
1,-1.891658,-0.867565,-0.824829,-1.366330,0.513682,0.702713,-1.754244,-0.182644,0.0,0.703361,...,1.186900,-0.470150,0.423304,-1.270108,0.771354,-1.267867,-0.140255,-1.042174,-0.371926,0
2,-1.968762,-0.883198,-0.091921,-0.577770,1.123162,0.731687,-1.280055,-0.249670,0.0,0.687387,...,1.376971,-0.470137,0.513959,-1.100738,0.955519,-1.097261,0.847781,-1.042107,-0.371820,0
3,-1.958116,-0.867475,0.693507,0.327115,1.636865,0.731687,-0.127985,-0.333661,0.0,0.667368,...,1.558821,-0.437744,0.705006,0.355822,-0.980130,0.350843,1.572762,-1.019584,-0.336060,0
4,-1.832527,-0.813155,1.524846,1.560727,1.028078,0.731687,1.359430,-0.355917,0.0,0.662064,...,1.780926,-0.383674,1.024098,0.396924,-0.972501,0.391854,1.653455,-0.990238,-0.289469,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12441,-0.026020,-0.622019,-0.519318,-0.979847,1.456856,-2.422136,-1.347590,1.901208,0.0,-1.641849,...,-1.108734,-0.143233,0.189478,-1.360548,0.723200,-1.357927,-0.354694,-0.918065,-0.174880,0
12442,-0.367893,-0.492759,-1.152625,-1.202467,1.280082,-1.971590,-1.612310,1.805874,0.0,-1.641849,...,-1.277789,-0.040691,0.099141,0.213483,-1.362938,0.206009,-0.194622,-0.844885,-0.058695,0
12443,-0.643058,-0.356696,-1.844844,-2.000190,-0.066119,-1.295771,-1.732637,1.963771,0.0,-1.641849,...,0.735496,-0.340993,0.014118,0.000592,-1.492194,-0.005821,-0.347161,-0.965277,-0.249839,0
12444,-0.968254,-0.519972,-1.285178,-2.853569,-1.439516,-1.521044,-2.310208,1.895249,0.0,-1.641849,...,0.796970,-0.729189,2.778709,0.070801,-1.547303,0.064038,-0.283132,-1.130522,-0.512194,0


In [58]:
df_test.drop(columns=['index','ATT_FLAG',
       'AttackID'],inplace=True)

In [59]:
test_cols=df_test.columns

In [60]:
scaler=StandardScaler()
df_test=scaler.fit_transform(df_test)
df_test=pd.DataFrame(df_test,columns=test_cols)
df_test


,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,F_PU2,...,P_J300,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422
0,0.583548,-0.555567,-0.883378,-2.366840,-1.535844,-2.328066,-0.551651,-0.586532,0.062366,0.592635,...,-1.937428,0.791420,-1.933359,-0.001283,-1.596200,0.538146,-1.636786,2.099018,1.240824,-2.037462
1,0.704883,-1.158800,-0.294490,-2.384901,-1.046705,-0.824117,-0.199940,-0.732843,0.062366,0.551845,...,-2.158465,0.971262,-2.159461,0.088967,0.913851,1.136353,0.931409,-2.335489,1.545753,-2.285204
2,0.850484,-1.748166,0.353287,-1.969502,-0.326583,0.564143,0.188155,-0.834842,0.062366,0.523408,...,-2.418096,0.903245,-2.417359,0.213224,1.051832,1.215117,1.069462,-2.184041,1.706687,-2.566728
3,0.826218,-1.872972,1.015787,-1.120644,0.434301,0.737676,0.794553,-0.454435,0.062366,0.629463,...,-0.769088,1.364379,-0.756921,0.434271,0.688616,1.183212,0.706057,-1.794363,-1.063084,-0.618580
4,0.777684,-1.630292,1.531064,-0.235665,1.249533,-0.072143,1.801174,-0.451926,0.062366,0.629929,...,-0.095451,-0.378707,-0.113944,0.680170,-1.320237,0.892085,-1.354589,2.960054,-0.753920,-0.070546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2576,-0.686423,-0.874518,-0.780323,-1.084523,-1.372798,-2.385910,-1.461248,1.654947,0.062366,-1.579977,...,-0.270877,-1.077325,-0.290586,0.158290,-1.304004,0.593978,-1.309925,-0.351362,-0.897915,-0.325795
2577,-1.009983,-0.749711,-1.413377,-1.283192,-0.924420,-2.038845,-1.764448,1.841389,0.062366,-1.579977,...,-0.295437,-1.282530,-0.315316,0.036648,-1.555618,0.539143,-1.561669,-0.322434,-0.925443,-0.374592
2578,-1.390165,-0.770512,-1.781432,-2.258475,-0.557565,-1.460403,-1.837215,1.858946,0.062366,-1.579977,...,-0.642781,0.883647,-0.626206,0.121667,-1.569822,0.580020,-1.575880,-0.296909,-1.041909,-0.581043
2579,-1.568123,-0.839849,-1.118933,-2.168171,-0.109187,-0.766273,-1.788703,-0.125027,0.062366,0.721300,...,-0.502440,0.972415,-0.488425,0.160906,-1.157905,0.691685,-1.163751,-0.334345,-0.933913,-0.389607


In [71]:
attack_sensors = [0]*15
attack_sensors

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [73]:
for sens in range(len(df_normal.columns)-1):
    X = df_normal.iloc[:,sens].values
    X = hankel.fit(X,60,0.5)
    X=X.T 
    kmeans,optimal_k = cluster.fit(X)
    # optimal_k = 16
    # kmeans = KMeans(n_clusters=optimal_k, init='k-means++')
    kmeans.fit(X)
    print(optimal_k)
    radiis_normal = []
    weights=[]
    centers=[]
    clusters_R=[]
    clusters_V=[]
    for i in range(optimal_k):
        cluster_ = X[np.where(kmeans.labels_ == i)[0]]
        r = rank.fit(cluster_)
        clusters_R.append(r)
        if(optimal_k==1):
            print("r: "+str(r))
        U,Sigma,VT = np.linalg.svd(cluster_)
        V = VT.T
        clusters_V.append(V)
        cluster_ = np.matmul(cluster_,V[:,:r])
        weight,center = meepc.fit(cluster_)
        weights.append(weight)
        centers.append(center)
        var1=np.square(cluster_-center)
        var2=np.matmul(weight,var1.T)
        radiis_normal.append(np.amax(np.sqrt(var2)))
    # X_attack = df_attack.iloc[:,sens].values
    # X_att = hankel.fit(X_attack,1,0.5)
    # X_att=X_att.T 
    # radiis_attack = []
    # for i in range(optimal_k):
    #     cluster_ = np.matmul(X_att,clusters_V[i][:,:clusters_R[i]])
    #     var1=np.square(cluster_-centers[i])
    #     var2=np.matmul(weights[i],var1.T)
    #     radiis_attack.append(np.sqrt(var2))
    # threshold_clusters = [0]*optimal_k
    # for i in range(optimal_k):
    #     threshold_clusters[i] = find_threshold(radiis_normal[i],radiis_attack[i])


    X_test=df_test.iloc[:,sens].values
    Xtest=hankel.fit(X_test,60,0.5)  #size -> 1800x274
    Xtest=Xtest.T  
    distances=[]
    for i in range(optimal_k):
        Xtest_cluster=np.matmul(Xtest,clusters_V[i][:,:clusters_R[i]]) 
        var1=np.square(Xtest_cluster-centers[i])
        var2=np.matmul(weights[i],var1.T)
        distance=np.sqrt(var2)
        distances.append(distance)

    stacked_arr=np.vstack(distances)
    distances_arr=np.transpose(stacked_arr)
    check_anamoly=radiis_normal-distances_arr     # positive = inside cluster | negative = outside cluster  # size --> 68 X 3
    #check for if any datapoint is coming inside the cluster
    is_positive=np.any(check_anamoly>0,axis=1)
    #indices of those rows
    positive_indices=np.where(is_positive)[0]
    # # for i in [[267, 268, 269, 272, 273]]:
    # #     print(distances_arr[i])
    cnt=0
    # print("sensor ->",sens)
    unique_s = set()
    for i in range(len(Xtest)):
        if i not in positive_indices:
                if i in attack_dict:
                    for j in attack_dict[i]:
                        if j != -1:
                            unique_s.add(j)
                print("predicted attack",i,end=' ')
                if i in list(attackindices):
                    cnt+=1
    # print("count",cnt)

    print(unique_s)
    for i in unique_s:
        attack_sensors[i] += 1
    print(attack_sensors)
    print("\nnext")
    # print(threshold_clusters)
    # print(distances_arr)
print(attack_sensors)

3
predicted attack 2 predicted attack 3 predicted attack 6 predicted attack 11 predicted attack 12 predicted attack 15 predicted attack 17 predicted attack 25 predicted attack 41 predicted attack 44 predicted attack 46 predicted attack 47 predicted attack 48 predicted attack 54 predicted attack 56 predicted attack 57 predicted attack 60 predicted attack 62 predicted attack 67 predicted attack 69 predicted attack 78 predicted attack 81 predicted attack 83 {2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14}
[0, 27, 31, 38, 35, 30, 32, 33, 27, 27, 29, 30, 29, 29, 24]

next
8
predicted attack 0 predicted attack 1 predicted attack 2 predicted attack 7 predicted attack 11 predicted attack 12 predicted attack 13 predicted attack 16 predicted attack 27 predicted attack 28 predicted attack 36 predicted attack 37 predicted attack 45 predicted attack 46 predicted attack 48 predicted attack 50 predicted attack 56 predicted attack 78 {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
[0, 28, 32, 39, 36, 31, 33, 34, 28,

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 2, in where
KeyboardInterrupt: 


8
predicted attack 14 predicted attack 44 predicted attack 47 {10, 11, 7}
[0, 30, 35, 42, 39, 33, 35, 37, 31, 31, 32, 33, 33, 32, 26]

next
7
predicted attack 9 predicted attack 15 predicted attack 30 predicted attack 31 predicted attack 46 predicted attack 47 predicted attack 58 predicted attack 74 predicted attack 75 {5, 6, 7, 10, 11, 12}
[0, 30, 35, 42, 39, 34, 36, 38, 31, 31, 33, 34, 34, 32, 26]

next
4
predicted attack 2 predicted attack 3 predicted attack 4 predicted attack 5 predicted attack 10 predicted attack 40 predicted attack 46 predicted attack 47 predicted attack 48 predicted attack 51 predicted attack 64 predicted attack 77 {2, 3, 4, 6, 10, 11}
[0, 30, 36, 43, 40, 34, 37, 38, 31, 31, 34, 35, 34, 32, 26]

next
1
r: 0
predicted attack 0 predicted attack 1 predicted attack 2 predicted attack 3 predicted attack 4 predicted attack 5 predicted attack 6 predicted attack 7 predicted attack 8 predicted attack 9 predicted attack 10 predicted attack 11 predicted attack 12 predicted

In [ ]:
array([ 8,  9, 10, 11, 12, 20, 21, 22, 23, 27, 
28, 29, 30, 31, 32, 39, 40,
 41, 42, 43, 44, 51, 52, 53, 54, 55, 63, 64, 65])

NameError: name 'array' is not defined